# Gathering and Sanitizing Netflix User Data

A concern when sharing information online is giving out more information than might be safe. Since this report is going to be about Netflix profile watch data the chances of that being true here is small, but not null. After we pull the data we will check the information included in the base file, then sanitized and/or [hash](https://www.techtarget.com/searchdatamanagement/definition/hashing)  the data.

To download the usage information from Netflix navigate to the url `https://www.netflix.com/account/getmyinfo` and submit a request for a copy of your information. After a short period of time you will receive an email from Netflix with a CSV file of your data.

Download and unzip the file. Inside are 11 folders and two files:

![netflix-report-tree structure](./files/assets/netflix-report-tree.png 'netflix report tree')

The report that we are looking for is `./CONTENT_INTERACTION/ViewingActivity.csv` and we can move that into the `files/non-sanitized` folder


We can now pull in the data and check what the columns are.

The two columns that I might be  about releasing are `` and ``. Looking at the actual data in the columns the only one that I will be sanitizing is ``.
First we has the values in the column, then we can check the DataFrame Header to check that there isn't any more data to sanitize.

In [2]:
df[''] = d[''].apply(hash)
print(df)

NameError: name 'd' is not defined

Now we save the data. To maintain data integrity we save the file in a different location

In [3]:
df.to_csv(r'.files/ViewingActivity_sanitized.csv')

NameError: name 'df' is not defined